In [2]:
!pip install numpy pandas scikit-learn tqdm scipy openai

In [1]:
import tensorflow as tf

# Print the TensorFlow version
print(tf.__version__)


2.15.0


In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
import scipy as sp
from concurrent.futures import ThreadPoolExecutor

In [4]:
! pip install -U LibRecommender

In [5]:
def abs_loss(gt_ratings, pred_ratings, isbn_to_index):
    loss = 0
    count = 0

    for _, row in gt_ratings.iterrows():
        user_id = row['User-ID']
        isbn = row['ISBN']
        gt_rating = row['Rating']

        item_index = isbn_to_index.get(isbn)
        if item_index is None or user_id not in pred_ratings:
            continue

        pred_rating = pred_ratings[user_id].get(item_index)
        if pred_rating is None:
            continue

        loss += np.abs(gt_rating - pred_rating)
        count += 1

    return loss / count if count > 0 else 0


def l2_loss(gt_ratings, pred_ratings, isbn_to_index):
    loss = 0
    count = 0

    for _, row in gt_ratings.iterrows():
        user_id = row['User-ID']
        isbn = row['ISBN']
        gt_rating = row['Rating']

        item_index = isbn_to_index.get(isbn)
        if item_index is None or user_id not in pred_ratings:
            continue

        pred_rating = pred_ratings[user_id].get(item_index)
        if pred_rating is None:
            continue

        loss += (gt_rating - pred_rating) ** 2
        count += 1

    return np.sqrt(loss / count) if count > 0 else 0

def dcg(relevances, rank=10):
    relevances = np.asarray(relevances)[:rank]
    n_relevances = len(relevances)
    if n_relevances == 0:
        return 0.

    discounts = np.log2(np.arange(n_relevances) + 2)
    return np.sum(relevances / discounts)


def ndcg(relevances, rank=10):
    best_dcg = dcg(sorted(relevances, reverse=True), rank)
    if best_dcg == 0:
        return 0.

    return dcg(relevances, rank) / best_dcg


def precision_at_k(recommended_items, relevant_items, k):
    recommended_at_k = recommended_items[:k]
    relevant_and_recommended = [item for item in recommended_at_k if item in relevant_items]
    return len(relevant_and_recommended) / k


def recall_at_k(recommended_items, relevant_items, k):
    """Calculate recall at k."""
    if relevant_items.size == 0:  # Use size attribute to check if the array is empty
        return 0.0  # Return 0 recall if there are no relevant items

    recommended_at_k = recommended_items[:k]
    relevant_and_recommended = [item for item in recommended_at_k if item in relevant_items]
    return len(relevant_and_recommended) / len(relevant_items)



In [6]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
books = pd.read_csv('/content/drive/MyDrive/Books_Extended.csv', delimiter=';', low_memory=False)
ratings = pd.read_csv('/content/drive/MyDrive/Ratings.csv', delimiter=';')
users = pd.read_csv('/content/drive/MyDrive/Users.csv', delimiter=';')

<ipython-input-7-965afd468f24>:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  users = pd.read_csv('/content/drive/MyDrive/Users.csv', delimiter=';')


In [8]:
df_prep_step_1 = pd.merge(books, ratings.query("`Rating` > 0"), on='ISBN', how='inner')

# join users data
df_prep_step_2 = pd.merge(df_prep_step_1, users, on='User-ID', how='inner')

In [9]:
df_prep = df_prep_step_2.drop(['Publisher'], axis=1)
df_isbn = df_prep.drop_duplicates()

In [10]:
df_prep = df_prep_step_2.drop(['Publisher'], axis=1)

In [11]:
df_isbn['Original_NaN'] = df_isbn['Age'].isna()

# Convert 'Age' to numeric, turning non-numeric values into NaN
df_isbn['Age'] = pd.to_numeric(df_isbn['Age'], errors='coerce')

# Drop rows where 'Age' is NaN and were not originally NaN
df_isbn = df_isbn[~(users['Age'].isna() & ~df_isbn['Original_NaN'])]

# Drop the 'Original_NaN' column as it's no longer needed
df_isbn.drop(columns=['Original_NaN'], inplace=True)

<ipython-input-11-a229782017b2>:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_isbn = df_isbn[~(users['Age'].isna() & ~df_isbn['Original_NaN'])]
<ipython-input-11-a229782017b2>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_isbn.drop(columns=['Original_NaN'], inplace=True)


In [12]:
df_isbn['User-ID'] = pd.to_numeric(df_isbn['User-ID'], errors='coerce')

# Drop rows where 'Age' is NaN and were not originally NaN
df_isbn = df_isbn[~(df_isbn['User-ID'].isna())]

In [13]:
age_outliers = df_isbn.query("Age > 100 or Age < 6")

user_outliers = age_outliers["User-ID"].to_list()

In [14]:
# exclude age outliers
df_isbn = df_isbn[~df_isbn["User-ID"].isin(user_outliers)]

In [15]:
unique_descriptions = df_isbn['Description'].unique()

In [15]:
from openai import OpenAI
client = OpenAI(api_key='key')

def get_embedding(text, model="text-embedding-3-small"):
    if text is None or pd.isna(text):
        return None
    text = text.replace("\n", " ")
    return client.embeddings.create(input=[text], model=model).data[0].embedding

In [17]:
description_to_embedding = {}

for description in tqdm(unique_descriptions, desc="Generating Embeddings"):
    if description not in description_to_embedding:
        embedding = get_embedding(description)
        description_to_embedding[description] = embedding

df_isbn['embedding'] = df_isbn['Description'].map(description_to_embedding)


Generating Embeddings: 100%|██████████| 38475/38475 [2:16:05<00:00,  4.71it/s]


In [33]:
def average_precision(predicted_items, actual_items, m):
    m = min(m, len(actual_items))
    ap = 0
    for i in range(m):
        ap += precision_at_k(predicted_items, actual_items, i + 1) / m
    return ap

In [19]:
df_isbn['embedding'].notna().sum()

79833

In [16]:
df_clened_with_emb = pd.read_csv('/content/df_with_emb_cleaned.csv')

In [323]:
df_cleaned.rename(columns={"User-ID": "user"}, inplace=True)

<ipython-input-323-939667d10c5d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned.rename(columns={"User-ID": "user"}, inplace=True)


In [324]:
df_cleaned.rename(columns={"ISBN": "item"}, inplace=True)

<ipython-input-324-24d637dee9f4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned.rename(columns={"ISBN": "item"}, inplace=True)


In [325]:
df_cleaned.rename(columns={"Rating": "label"}, inplace=True)

<ipython-input-325-21c57fff3936>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned.rename(columns={"Rating": "label"}, inplace=True)


In [18]:
pca_columns = [f'pca_dim_{i+1}' for i in range(10)]

In [19]:
columns = ['user', 'item', 'label']
user_cols = ['Age']
item_cols = pca_columns + ['Year']

columns = columns + user_cols + item_cols

In [91]:
from libreco.algorithms import NCF
from sklearn.model_selection import train_test_split
from tensorflow.compat.v1 import reset_default_graph

from libreco.data import DataInfo, DatasetFeat, split_by_ratio
train_df, test_df = train_test_split(
    df_clened_with_emb,
    test_size=0.3,
    random_state=42
)

eval_user_mask = test_df['user'].isin(train_df['user'].unique())
eval_item_mask = test_df['item'].isin(train_df['item'].unique())

eval_df = test_df[eval_user_mask & eval_item_mask]

train_data, data_info = DatasetFeat.build_trainset(train_df, user_col=user_cols, item_col=item_cols, dense_col=item_cols + user_cols)
test_data = DatasetFeat.build_testset(eval_df)

reset_default_graph()
ncf = NCF(
    task="rating",
    data_info=data_info,
    n_epochs=400,
    embed_size=512,
    lr=0.5,
    use_bn=True,
    lr_decay=True,
    reg=0.01,
    seed=42,
    dropout_rate=0.2
)

ncf.fit(train_data, verbose=2, neg_sampling=False, eval_data=test_data, metrics=['rmse','mae'])

Training start time: 2024-08-11 17:38:30


/usr/local/lib/python3.10/dist-packages/libreco/layers/dense.py:31: UserWarning: `tf.layers.batch_normalization` is deprecated and will be removed in a future version. Please use `tf.keras.layers.BatchNormalization` instead. In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.BatchNormalization` documentation).
  net = tf.layers.batch_normalization(net, training=is_training)
/usr/local/lib/python3.10/dist-packages/libreco/layers/dense.py:39: UserWarning: `tf.layers.batch_normalization` is deprecated and will be removed in a future version. Please use `tf.keras.layers.BatchNormalization` instead. In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.BatchNormalization` documentation).
  net = tf.layers.batch_normalization(net, training=is_training)
/usr/local/lib/python3.10/dist-packages/libreco/layers/dense.py:47: UserWarning: `tf.layers.dropout` is deprecated and

With lr_decay, epoch 1 learning rate: 0.5


train: 100%|██████████| 57/57 [00:01<00:00, 31.65it/s]


Epoch 1 elapsed: 1.805s
	 train_loss: 12997.708


...

	 eval rmse: 1.7166
	 eval mae: 1.3704
With lr_decay, epoch 400 learning rate: 3.170199036617305e-08


train: 100%|██████████| 57/57 [00:00<00:00, 294.61it/s]


Epoch 400 elapsed: 0.197s
	 train_loss: 3.4911


eval_pointwise: 100%|██████████| 1/1 [00:00<00:00, 300.67it/s]

	 eval rmse: 1.7140
	 eval mae: 1.3692


In [92]:
predictions = {}

In [93]:
predictions = {}
for user_id in tqdm(eval_df['user'].unique()):
    predictions[user_id] = ncf.recommend_user(user=user_id, n_rec=20)[user_id]

100%|██████████| 273/273 [00:00<00:00, 510.28it/s]


In [94]:
mean_ap = 0
total_abs_loss = 0
total_l2_loss = 0
total_precision_at_k = 0
total_recall_at_k = 0
total_ndcg = 0
total_dcg = 0

count_users_with_relevant_items = 0
count = 0

for user_id in eval_df['user'].unique():
    relevant_items = eval_df[eval_df['user']==user_id]['item'].to_numpy()

    predicted_isbns = predictions.get(user_id, [])

    if predicted_isbns is None:
        continue

    ap = average_precision(predicted_isbns, relevant_items, m=20)
    mean_ap += ap

    precision = precision_at_k(predicted_isbns, relevant_items, k=20)
    recall = recall_at_k(predicted_isbns, relevant_items, k=20)
    total_precision_at_k += precision
    total_recall_at_k += recall

    count_users_with_relevant_items += 1

mean_ap = mean_ap / count_users_with_relevant_items if count_users_with_relevant_items > 0 else 0
mean_precision_at_k = total_precision_at_k / count_users_with_relevant_items if count_users_with_relevant_items > 0 else 0
mean_recall_at_k = total_recall_at_k / count_users_with_relevant_items if count_users_with_relevant_items > 0 else 0

print(f"Mean Average Precision: {mean_ap}")
print(f"Mean Precision at K: {mean_precision_at_k}")
print(f"Mean Recall at K: {mean_recall_at_k}")

Mean Average Precision: 0.006588319088319089
Mean Precision at K: 0.002380952380952381
Mean Recall at K: 0.02033403104831676
